In [1]:
import math
import multiprocessing as mg
import multiprocessing.pool
# import pys2let as ps
import random
import string
import itertools
import os

import jax
jax.config.update("jax_enable_x64", True)
import s2fft
import healpy as hp
import numpy as np
import s2wav
import s2wav
import matplotlib.pyplot as plt
%matplotlib inline 
import skyclean
from skyclean import CMB_data

In [2]:
def mw_alm_2_hp_alm(MW_alm, lmax):
    '''MW_alm: 2D array of shape (Lmax, 2*Lmax-1) (MW sampling, McEwen & Wiaux)
    '''
    # Initialize the 1D hp_alm array with the appropriate size
    hp_alm = np.zeros(hp.Alm.getsize(lmax), dtype=np.complex128)
        
    for l in range(lmax + 1):
        for m in range(-l, l + 1):
            index = hp.Alm.getidx(lmax, l, abs(m))
            if m < 0:
                hp_alm[index] = (-1)**m * np.conj(MW_alm[l, lmax + m])
            else:
                hp_alm[index] = MW_alm[l, lmax + m]

    return hp_alm


def Single_Map_doubleworker(MW_Pix_Map):
    '''
    Input: MW_Pix_Map: list of mw maps at different scales 
    Each pixel map is a wavelet pixel map of shape (1, Lmax, 2*Lmax-1) (MW sampling, McEwen & Wiaux)
    It is the output of s2wav.analysis
    (Scale: 0, size (1, 4, 7))

    Process:
    1. Covert MW Pixel Map to MW alm space using s2fft.forward

    2. Add zero to the mw alms  (Is it correct? or should I add zeros to the hp alm's and then convert to mw alm's)
    by adding zeros to the MW alm's we are increasing the resolution of the map
    Double the rows of the mw alms, since, the number of rows represents the L (level of detail)
    
    3. Convert mw alm to mw map 
    
    '''
    # print("original Pixel Map size", MW_Pix_Map.shape)
    MW_alm = s2fft.forward(MW_Pix_Map, L = MW_Pix_Map.shape[1])
    # print("original alm size", MW_alm.shape)
    
   
    # print("Scale:",i,"original alm size", MW_alm[i].shape)
    padded_alm = np.zeros((MW_alm.shape[0]*2,MW_alm.shape[1]*2))
    # stored_wavelet_coeffs_alm_doubled.append(skyclean.double_resolution(stored_wavelet_coeffs_alm[i]))
    padded_alm[:MW_alm.shape[0], :MW_alm.shape[1]] = MW_alm
    # print("padded alm size", padded_alm.shape)
    MW_alm_doubled = padded_alm
    
    MW_Pix_Map_doubled = s2fft.inverse(MW_alm_doubled, L = MW_alm_doubled.shape[0])
    # print("Scale:","doubled map size", MW_Pix_Map_doubled.shape)

    return MW_Pix_Map_doubled


def smoothed_covariance(MW_Map1, MW_Map2):
    '''
    Input: MW_Map1, MW_Map2: same size MW pixel wavelet maps at different frequencies
    output: R_map: smoothed covariance map beteen MW_Map1 and MW_Map2
    '''
    smoothing_lmax = MW_Map1.shape[0]
    # Get the real part of the map
    map1 = np.real(MW_Map1)
    map2 = np.real(MW_Map2)
    # Covariance matrix
    R_MW_Pixel_map = np.multiply(map1,map2) + 0.j #Add back in zero imaginary part
    # print("R", R_MW_Pixel_map.shape)

    # smoothing in harmonic space for efficiency
    R_MW_alm = s2fft.forward(R_MW_Pixel_map, L = smoothing_lmax)
    # print("R_MW_alm", R_MW_alm.shape)


    nsamp = 1200.0
    lmax_at_scale_j = R_MW_alm.shape[0]
    npix = hp.nside2npix(1<<(int(0.5*lmax_at_scale_j)-1).bit_length())
    # (int(0.5*scale_lmax)-1).bit_length() calculates the number of bits necessary to represent the integer int(0.5*scale_lmax)-1 in binary.
    # 1 << (int(0.5*scale_lmax)-1).bit_length() performs a bitwise left shift, essentially calculating 2^(number of bits).
    scale_fwhm = 4.0 * math.sqrt(nsamp / npix)
    # for high resolution maps, it is still the same number pixels sampled by the actual range is smaller.
    # the beam will become very narrow.


    gauss_smooth = hp.gauss_beam(scale_fwhm,lmax=smoothing_lmax-1)
    MW_alm_beam_convolved = np.zeros(R_MW_alm.shape, dtype=np.complex128)

    # Convolve the MW alms with the beam
    for i in range(R_MW_alm.shape[1]):
        MW_alm_beam_convolved[:, i] = R_MW_alm[:, i] * gauss_smooth
    
    R_covariance_map = s2fft.inverse(MW_alm_beam_convolved, L = smoothing_lmax)

    return R_covariance_map




 

## Loaded mw wavelet coefficient map
stored_wavelet_coeffs_pix = [np.load(f"../convolution/wavelet_coefficient/wav_30_{i}.npy", allow_pickle=True) for i in range(12)]
stored_scaling_coeffs_pix = np.load("../convolution/scaling_coefficient/scal_30.npy")


# print(stored_wavelet_coeffs_pix[0].shape)
stored_wavelet_coeffs_pix = stored_wavelet_coeffs_pix[:3]

wavelet_MW_Pix_Map_doubled = Single_Map_doubleworker(stored_wavelet_coeffs_pix[0])

# display(wavelet_MW_Pix_Map_doubled[0])
# Why oen dimension is reduced?
# Is it the spin?
print(wavelet_MW_Pix_Map_doubled.shape)


R_covariance_map = smoothed_covariance(wavelet_MW_Pix_Map_doubled, wavelet_MW_Pix_Map_doubled)

R_covariance_map.shape

(8, 15)


(8, 15)

In [60]:
# smoothed_covariance(wavelet_MW_Pix_Map_doubled, wavelet_MW_Pix_Map_doubled)[0][0]

(1.9514297468690185e-11-4.36429717423837e-29j)

In [3]:
# we don't want to store all the data in a dictionary 
# wavelet is wavelet, and alm is alm 
# we get them from the same function

import numpy as np

def load_frequency_data(base_path, file_template, frequencies, scales=None):
    """
    Load NumPy arrays from dynamically generated file paths for each frequency and scale.
    
    Args:
        base_path (str): The base path where the files are located.
        file_template (str): The template for the file names, with placeholders for frequency and scale.
        frequencies (list): A list of frequency names.
        scales_: A lists of scales.
        
    Returns:
        dict: A dictionary where keys are tuples of (frequency, scale) and values are loaded NumPy arrays.
    """
    frequency_data = {}
    for frequency in frequencies:
        for scale in scales:
            # Generate the file path using the template and the current frequency and scale
            path = f"{base_path}/{file_template.format(frequency, scale)}"
            try:
                frequency_data[(frequency, scale)] = np.load(path, allow_pickle=True)
            except Exception as e:
                print(f"Error loading {path} for frequency {frequency} and scale {scale}: {e}")
    return frequency_data



base_path = "../convolution/wavelet_coefficient"
file_template = "wav_{}_{}.npy"
frequencies = ['030', '070']
scales = [0, 1, 2]

original_wavelet_c_j = load_frequency_data(base_path, file_template, frequencies, scales)

# for (frequency, scale), data in frequency_data.items():
#     print(f"Frequency: {frequency}, Scale: {scale}, Data shape: {data.shape}")




In [4]:
for i in frequencies:
    for j in scales:
        # print(frequency_data[(i,j)].shape)
        wavelet_MW_Pix_Map_doubled = Single_Map_doubleworker(original_wavelet_c_j[(i,j)])
        np.save(f"../convolution/wavelet_coefficient_doubled/wav_{i}_{j}.npy", wavelet_MW_Pix_Map_doubled)

In [56]:
# npix*npix*nfrequency

doubled_MW_wav_c_j = load_frequency_data("../convolution/wavelet_coefficient_doubled", "wav_{}_{}.npy", frequencies, scales)
print(doubled_MW_wav_c_j.keys())


# nfrequency*nfrequency*npix*npix


total_frequency = len(frequencies)
covariance_matrix_all_freq = np.full((len(scales),total_frequency, total_frequency), None)
# Go to each scale in each frequency
# for i in frequencies:
for j in scales:
    # calculate the covariance matrix
    for i in range(total_frequency):
        for fq in range(i, total_frequency):
            # print(f"Element at ({i}, {fq}): {convariance_matrix_all_freq[j, i, fq]}")
            covariance_matrix_all_freq[j, i, fq] = smoothed_covariance(doubled_MW_wav_c_j[(frequencies[i],j)], doubled_MW_wav_c_j[(frequencies[fq],j)])
            np.save(f"../convolution/covariance_matrix/cov_{j}_{frequencies[i]}_{frequencies[fq]}.npy", covariance_matrix_all_freq[j, i, fq])
            print(f"Element at (scale {j}, covariance of frequency {frequencies[i]} and frqeucny {frequencies[fq]}): {covariance_matrix_all_freq[j, i, fq].shape}")
            
    # Symmetric 
    # The outer loop starts from 1 (range(1, total_frequency)) to skip the diagonal element in the first row.
    # The inner loop runs from 0 to i-1 (range(i)) to exclude the diagonal and include only the lower triangular elements.

    for l1 in range(1, total_frequency):
        for l2 in range(l1):
            covariance_matrix_all_freq[j, l1, l2] = covariance_matrix_all_freq[j, l2, l1]

            # print(f"Element at ({i}, {j}): {matrix[i, j]}")
# symmetric_matrix = np.tril(convariance_matrix_all_freq[0, 1, 1]) + np.tril(convariance_matrix_all_freq[0, 1, 1], -1).T
# print(symmetric_matrix.shape)
print(covariance_matrix_all_freq.shape)
# convariance_matrix_all_freq[2, 0, 1]

dict_keys([('030', 0), ('030', 1), ('030', 2), ('070', 0), ('070', 1), ('070', 2)])
Element at (scale 0, covariance of frequency 030 and frqeucny 030): (8, 15)
Element at (scale 0, covariance of frequency 030 and frqeucny 070): (8, 15)
Element at (scale 0, covariance of frequency 070 and frqeucny 070): (8, 15)
Element at (scale 1, covariance of frequency 030 and frqeucny 030): (8, 15)
Element at (scale 1, covariance of frequency 030 and frqeucny 070): (8, 15)
Element at (scale 1, covariance of frequency 070 and frqeucny 070): (8, 15)
Element at (scale 2, covariance of frequency 030 and frqeucny 030): (16, 31)
Element at (scale 2, covariance of frequency 030 and frqeucny 070): (16, 31)
Element at (scale 2, covariance of frequency 070 and frqeucny 070): (16, 31)
(3, 2, 2)


In [70]:
n1 = smoothed_covariance(doubled_MW_wav_c_j[(frequencies[0],0)], doubled_MW_wav_c_j[(frequencies[0],0)])

In [71]:
n1.shape

(8, 15)

In [74]:
full_array[0, 0].shape

(8, 15)

In [78]:
full_array = np.zeros((total_frequency, total_frequency, 8, 15), dtype=np.complex128)


# calculate the covariance matrix
for i in range(total_frequency):
    for fq in range(i, total_frequency):
        print(i,fq)
        # print(f"Element at ({i}, {fq}): {convariance_matrix_all_freq[j, i, fq]}")
        print(full_array[i, fq].shape)
        full_array[i, fq] = smoothed_covariance(doubled_MW_wav_c_j[(frequencies[i],0)], doubled_MW_wav_c_j[(frequencies[fq],0)])
        np.save(f"../convolution/covariance_matrix/cov_{0}_{frequencies[i]}_{frequencies[fq]}.npy", full_array[i, fq])
        print(f"Element at ( covariance of frequency {frequencies[i]} and frqeucny {frequencies[fq]}): {full_array[i, fq]==convariance_matrix_all_freq[0, i, fq]}")
        
# Symmetric 
# The outer loop starts from 1 (range(1, total_frequency)) to skip the diagonal element in the first row.
# The inner loop runs from 0 to i-1 (range(i)) to exclude the diagonal and include only the lower triangular elements.

for l1 in range(1, total_frequency):
    for l2 in range(l1):
        full_array[l1, l2] = full_array[l2, l1]



0 0
(8, 15)
Element at ( covariance of frequency 030 and frqeucny 030): [[ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True]
 [ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True]
 [ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True]
 [ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True]
 [ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True]
 [ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True]
 [ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True]
 [ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True]]
0 1
(8, 15)
Element at ( covariance of frequency 030 and frqeucny 070): [[ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True]


In [58]:
full_array

array([[[[ 2.02320458e-10+2.51016797e-27j,
           2.02320458e-10+2.51016797e-27j,
           2.02320458e-10+2.51016797e-27j,
           2.02320458e-10+2.51016797e-27j,
           2.02320458e-10+2.51016797e-27j,
           2.02320458e-10+2.51016797e-27j,
           2.02320458e-10+2.51016797e-27j,
           2.02320458e-10+2.51016797e-27j,
           2.02320458e-10+2.51016797e-27j,
           2.02320458e-10+2.51016797e-27j,
           2.02320458e-10+2.51016797e-27j,
           2.02320458e-10+2.51016797e-27j,
           2.02320458e-10+2.51016797e-27j,
           2.02320458e-10+2.51016797e-27j,
           2.02320458e-10+2.51016797e-27j],
         [ 2.02320458e-10+2.51016797e-27j,
           2.02320458e-10+2.51016797e-27j,
           2.02320458e-10+2.51016797e-27j,
           2.02320458e-10+2.51016797e-27j,
           2.02320458e-10+2.51016797e-27j,
           2.02320458e-10+2.51016797e-27j,
           2.02320458e-10+2.51016797e-27j,
           2.02320458e-10+2.51016797e-27j,
          

In [57]:
covariance_matrix_all_freq[0,0,0]

array([[2.67902388e-41+1.66012882e-59j, 2.67902388e-41+1.66012907e-59j,
        2.67902388e-41+1.66012907e-59j, 2.67902388e-41+1.66012912e-59j,
        2.67902388e-41+1.66012920e-59j, 2.67902388e-41+1.66012886e-59j,
        2.67902388e-41+1.66012870e-59j, 2.67902388e-41+1.66012849e-59j,
        2.67902388e-41+1.66012828e-59j, 2.67902388e-41+1.66012814e-59j,
        2.67902388e-41+1.66012794e-59j, 2.67902388e-41+1.66012814e-59j,
        2.67902388e-41+1.66012822e-59j, 2.67902388e-41+1.66012829e-59j,
        2.67902388e-41+1.66012863e-59j],
       [2.67902389e-41+1.66012905e-59j, 2.67902389e-41+1.66012960e-59j,
        2.67902389e-41+1.66012984e-59j, 2.67902388e-41+1.66012951e-59j,
        2.67902388e-41+1.66012923e-59j, 2.67902388e-41+1.66012908e-59j,
        2.67902388e-41+1.66012846e-59j, 2.67902388e-41+1.66012773e-59j,
        2.67902388e-41+1.66012740e-59j, 2.67902388e-41+1.66012734e-59j,
        2.67902388e-41+1.66012717e-59j, 2.67902388e-41+1.66012712e-59j,
        2.67902388e-41+

In [54]:
full_array[0,0].shape

(8, 15)

In [22]:
# Covariance matrix for the first scale 
n_0 = np.array(convariance_matrix_all_freq[0,:,:])
print(n_0[0][0].shape)
n_0.shape

(8, 15)


(2, 2)

In [64]:
print(data[0][0])

[[1.67902388e-41+1.66012882e-59j 2.67902388e-41+1.66012907e-59j]
 [3.67902389e-41+1.66012905e-59j 4.67902389e-41+1.66012960e-59j]]


In [63]:
import numpy as np

# Example data block (simplified for demonstration)
# Let's assume we are handling a smaller block for ease of demonstration
data = [
    [
        np.array([
            [1.67902388e-41+1.66012882e-59j, 2.67902388e-41+1.66012907e-59j],
            [3.67902389e-41+1.66012905e-59j, 4.67902389e-41+1.66012960e-59j]
        ]),
        np.array([
            [-1.07929682e-42-5.39656298e-59j, -2.07929682e-42-5.39656297e-59j],
            [-3.07929683e-42-5.39656306e-59j, -4.07929683e-42-5.39656305e-59j]
        ])
    ],
    [
        np.array([
            [1.99167996e-43+2.64524657e-60j, 2.99167996e-43+2.64524646e-60j],
            [3.99167993e-43+2.64524672e-60j, 4.99167994e-43+2.64524652e-60j]
        ]),
        np.array([
            [1.99167996e-43+3.64524657e-60j, 2.99167996e-43+3.64524646e-60j],
            [3.99167993e-43+3.64524672e-60j, 4.99167994e-43+3.64524652e-60j]
        ])
    ]
]
import numpy as np

def create_4d_array_from_nested_data(data):
    """
    Converts a nested list structure containing complex-valued arrays into a structured 4D NumPy array.
    
    Args:
    data (list): A nested list of NumPy arrays with complex numbers.
    
    Returns:
    np.ndarray: A 4D NumPy array containing the structured data.
    """
    # Determine dimensions based on the first element's structure
    blocks_per_dim1 = len(data)
    blocks_per_dim2 = len(data[0])
    sub_block_rows = data[0][0].shape[0]
    sub_block_cols = data[0][0].shape[1]
    # print(blocks_per_dim1, blocks_per_dim2, sub_block_rows, sub_block_cols)
    # Create an empty 4D array with a complex data type
    full_array1 = np.zeros((blocks_per_dim1, blocks_per_dim2, sub_block_rows, sub_block_cols), dtype=np.complex128)
    # Fill the array with data from the nested structure
    for i in range(blocks_per_dim1):
        for j in range(blocks_per_dim2):
            print(i,j)
            # print(full_array1[i, j])
            full_array1[i, j] = data[i][j]
            # print(full_array1[i, j])
            print(type(data[i][j]))
            print(data[i][j].shape)
    return full_array1

create_4d_array_from_nested_data(data).shape

# # Create the 4D array using the function
# structured_array = create_4d_array_from_nested_data(data)
# print("Structured 4D Array:\n", structured_array)



0 0
<class 'numpy.ndarray'>
(2, 2)
0 1
<class 'numpy.ndarray'>
(2, 2)
1 0
<class 'numpy.ndarray'>
(2, 2)
1 1
<class 'numpy.ndarray'>
(2, 2)


(2, 2, 2, 2)

In [46]:
import numpy as np

# Define the complex number blocks
a = np.array([[1.67902388e-41 + 1.66012882e-59j, 2.67902388e-41 + 1.66012907e-59j],
              [3.67902389e-41 + 1.66012905e-59j, 4.67902389e-41 + 1.66012960e-59j]])

b = np.array([[-1.07929682e-42 - 5.39656298e-59j, -2.07929682e-42 - 5.39656297e-59j],
              [-3.07929683e-42 - 5.39656306e-59j, -4.07929683e-42 - 5.39656305e-59j]])

c = np.array([[1.99167996e-43 + 2.64524657e-60j, 2.99167996e-43 + 2.64524646e-60j],
              [3.99167993e-43 + 2.64524672e-60j, 4.99167994e-43 + 2.64524652e-60j]])

d = np.array([[1.99167996e-43 + 3.64524657e-60j, 2.99167996e-43 + 3.64524646e-60j],
              [3.99167993e-43 + 3.64524672e-60j, 4.99167994e-43 + 3.64524652e-60j]])

# Stack these blocks into a 4D array
# We arrange these blocks into a larger 2x2 grid of 2x2 blocks
data = np.array([[a, b], [c, d]])

print("Created 4D Array:\n", data)
print("Shape of 4D Array:", data.shape)


Created 4D Array:
 [[[[ 1.67902388e-41+1.66012882e-59j  2.67902388e-41+1.66012907e-59j]
   [ 3.67902389e-41+1.66012905e-59j  4.67902389e-41+1.66012960e-59j]]

  [[-1.07929682e-42-5.39656298e-59j -2.07929682e-42-5.39656297e-59j]
   [-3.07929683e-42-5.39656306e-59j -4.07929683e-42-5.39656305e-59j]]]


 [[[ 1.99167996e-43+2.64524657e-60j  2.99167996e-43+2.64524646e-60j]
   [ 3.99167993e-43+2.64524672e-60j  4.99167994e-43+2.64524652e-60j]]

  [[ 1.99167996e-43+3.64524657e-60j  2.99167996e-43+3.64524646e-60j]
   [ 3.99167993e-43+3.64524672e-60j  4.99167994e-43+3.64524652e-60j]]]]
Shape of 4D Array: (2, 2, 2, 2)


In [45]:
print(2*2*8*15)
sum(sum(sum(sum(np.swapaxes(np.swapaxes(arr,0,2),1,3)==rearrange_blocks(create_4d_array_from_nested_data(n_0))))))

480
0 0
0 1
1 0
1 1


480

In [34]:
print(rearrange_blocks(create_4d_array_from_nested_data(n_0)))   

0 0
0 1
1 0
1 1
[[[[ 2.67902388e-41+1.66012882e-59j -2.07929682e-42-5.39656298e-59j]
   [-2.07929682e-42-5.39656298e-59j  5.99167996e-43+2.64524657e-60j]]

  [[ 2.67902388e-41+1.66012907e-59j -2.07929682e-42-5.39656297e-59j]
   [-2.07929682e-42-5.39656297e-59j  5.99167996e-43+2.64524646e-60j]]

  [[ 2.67902388e-41+1.66012907e-59j -2.07929682e-42-5.39656297e-59j]
   [-2.07929682e-42-5.39656297e-59j  5.99167996e-43+2.64524643e-60j]]

  [[ 2.67902388e-41+1.66012912e-59j -2.07929682e-42-5.39656296e-59j]
   [-2.07929682e-42-5.39656296e-59j  5.99167996e-43+2.64524638e-60j]]

  [[ 2.67902388e-41+1.66012920e-59j -2.07929682e-42-5.39656295e-59j]
   [-2.07929682e-42-5.39656295e-59j  5.99167997e-43+2.64524628e-60j]]

  [[ 2.67902388e-41+1.66012886e-59j -2.07929681e-42-5.39656293e-59j]
   [-2.07929681e-42-5.39656293e-59j  5.99167997e-43+2.64524634e-60j]]

  [[ 2.67902388e-41+1.66012870e-59j -2.07929681e-42-5.39656292e-59j]
   [-2.07929681e-42-5.39656292e-59j  5.99167997e-43+2.64524635e-60j]]

  [[

IndexError: index 1 is out of bounds for axis 0 with size 1

In [ ]:
def ILC(MW_Pix_Map):

    
    Current_Wavelet_Map = MW_Pix_Map
    
    # Define the size of the smoothing beam 
    # 1200 pixels, size of the sphere
    nsamp = 1200.0
    lmax_at_scale_j = Current_Wavelet_Map.shape[0]
    npix = hp.nside2npix(1<<(int(0.5*lmax_at_scale_j)-1).bit_length())
    # (int(0.5*scale_lmax)-1).bit_length() calculates the number of bits necessary to represent the integer int(0.5*scale_lmax)-1 in binary.
    # 1 << (int(0.5*scale_lmax)-1).bit_length() performs a bitwise left shift, essentially calculating 2^(number of bits).
    scale_fwhm = 4.0 * math.sqrt(nsamp / npix)
    # for high resolution maps, it is still the same number pixels sampled by the actual range is smaller.
    # the beam will become very narrow.
    print(lmax_at_scale_j,npix, scale_fwhm)


    # after smoothworker, 81 covariance maps 


    

# ILC(wavelet_MW_Pix_Map_doubled)

# print(wavelet_MW_Pix_Map_doubled.shape)
    

